# Main Experiment

The Orchestrator (Run experiments & Visualize here)

**Refactored for Step-by-Step Debugging**

In [ ]:
# --- CELL 1: Imports & Setup ---
import torch
import time
import pandas as pd
import matplotlib.pyplot as plt

from src.data_utils import get_split_cifar10
from src.models import get_resnet18
from src.trainer import train_baseline, train_constrained, evaluate
# Updated imports to include MagnitudePruningProjector and remove NMF
from src.decompositions import SVDProjector, QRProjector, RSVDProjector, MagnitudePruningProjector

# CONFIGURATION
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS_A = 15 # [cite: 26]
EPOCHS_B = 10
BATCH_SIZE = 64

# Data Setup [cite: 3]
print("--- Phase 1: Data Setup ---")
train_A, test_A, train_B, test_B = get_split_cifar10(BATCH_SIZE)

# Initialize global results container
results = []
print("Setup Complete. Results list initialized.")

In [ ]:
# --- CELL 2: Baseline Training (Task A) ---
print("--- Phase 2: Training Expert Model (Task A) ---")
model_expert = get_resnet18(num_classes=10).to(DEVICE)
model_expert = train_baseline(model_expert, train_A, EPOCHS_A, DEVICE)

# Checkpoint the expert [cite: 29]
torch.save(model_expert.state_dict(), "checkpoints/task_a_expert.pth")
acc_A_initial = evaluate(model_expert, test_A, DEVICE)
print(f"Task A Expert Accuracy: {acc_A_initial:.2f}%")

In [ ]:
# --- CELL 3: Experiment 1 - Naive Baseline ---
print("\n--- Experiment: Naive ---")

# 1. Reload Clean Weights
model_current = get_resnet18(num_classes=10).to(DEVICE)
model_current.load_state_dict(torch.load("checkpoints/task_a_expert.pth"))

# 2. Train without constraints
model_current = train_baseline(model_current, train_B, EPOCHS_B, DEVICE)

# 3. Metrics
acc_A_final = evaluate(model_current, test_A, DEVICE)
acc_B_final = evaluate(model_current, test_B, DEVICE)
forgetting = acc_A_initial - acc_A_final

results.append({
    "Method": "Naive",
    "Task A Acc (Retention)": acc_A_final,
    "Task B Acc (Plasticity)": acc_B_final,
    "Forgetting": forgetting,
    "Prep Time (s)": 0
})
print(f"Naive Result: Retention={acc_A_final:.2f}%, Forgetting={forgetting:.2f}%")

In [ ]:
# --- CELL 4: Experiment 2 - Standard SVD ---
print("\n--- Experiment: SVD ---")

# 1. Reload Clean Weights
model_current = get_resnet18(num_classes=10).to(DEVICE)
model_current.load_state_dict(torch.load("checkpoints/task_a_expert.pth"))

# 2. Subspace Extraction
projector = SVDProjector()
start_time = time.time()
projector.compute_subspaces(model_current)
prep_time = time.time() - start_time
print(f"Subspace computed in {prep_time:.2f}s")

# 3. Constrained Training
model_current = train_constrained(model_current, train_B, EPOCHS_B, DEVICE, projector)

# 4. Metrics
acc_A_final = evaluate(model_current, test_A, DEVICE)
acc_B_final = evaluate(model_current, test_B, DEVICE)
forgetting = acc_A_initial - acc_A_final

results.append({
    "Method": "SVD",
    "Task A Acc (Retention)": acc_A_final,
    "Task B Acc (Plasticity)": acc_B_final,
    "Forgetting": forgetting,
    "Prep Time (s)": prep_time
})
print(f"SVD Result: Retention={acc_A_final:.2f}%, Forgetting={forgetting:.2f}%")

In [ ]:
# --- CELL 5: Experiment 3 - QR Decomposition ---
print("\n--- Experiment: QR ---")

# 1. Reload Clean Weights
model_current = get_resnet18(num_classes=10).to(DEVICE)
model_current.load_state_dict(torch.load("checkpoints/task_a_expert.pth"))

# 2. Subspace Extraction
projector = QRProjector()
start_time = time.time()
projector.compute_subspaces(model_current)
prep_time = time.time() - start_time
print(f"Subspace computed in {prep_time:.2f}s")

# 3. Constrained Training
model_current = train_constrained(model_current, train_B, EPOCHS_B, DEVICE, projector)

# 4. Metrics
acc_A_final = evaluate(model_current, test_A, DEVICE)
acc_B_final = evaluate(model_current, test_B, DEVICE)
forgetting = acc_A_initial - acc_A_final

results.append({
    "Method": "QR",
    "Task A Acc (Retention)": acc_A_final,
    "Task B Acc (Plasticity)": acc_B_final,
    "Forgetting": forgetting,
    "Prep Time (s)": prep_time
})
print(f"QR Result: Retention={acc_A_final:.2f}%, Forgetting={forgetting:.2f}%")

In [ ]:
# --- CELL 6: Experiment 4 - Randomized SVD ---
print("\n--- Experiment: RSVD ---")

# 1. Reload Clean Weights
model_current = get_resnet18(num_classes=10).to(DEVICE)
model_current.load_state_dict(torch.load("checkpoints/task_a_expert.pth"))

# 2. Subspace Extraction
projector = RSVDProjector()
start_time = time.time()
projector.compute_subspaces(model_current)
prep_time = time.time() - start_time
print(f"Subspace computed in {prep_time:.2f}s")

# 3. Constrained Training
model_current = train_constrained(model_current, train_B, EPOCHS_B, DEVICE, projector)

# 4. Metrics
acc_A_final = evaluate(model_current, test_A, DEVICE)
acc_B_final = evaluate(model_current, test_B, DEVICE)
forgetting = acc_A_initial - acc_A_final

results.append({
    "Method": "RSVD",
    "Task A Acc (Retention)": acc_A_final,
    "Task B Acc (Plasticity)": acc_B_final,
    "Forgetting": forgetting,
    "Prep Time (s)": prep_time
})
print(f"RSVD Result: Retention={acc_A_final:.2f}%, Forgetting={forgetting:.2f}%")

In [ ]:
# --- CELL 7: Experiment 5 - Magnitude Pruning ---
print("\n--- Experiment: Magnitude Pruning ---")

# 1. Reload Clean Weights
model_current = get_resnet18(num_classes=10).to(DEVICE)
model_current.load_state_dict(torch.load("checkpoints/task_a_expert.pth"))

# 2. Subspace Extraction
projector = MagnitudePruningProjector()
start_time = time.time()
projector.compute_subspaces(model_current)
prep_time = time.time() - start_time
print(f"Subspace computed in {prep_time:.2f}s")

# 3. Constrained Training
model_current = train_constrained(model_current, train_B, EPOCHS_B, DEVICE, projector)

# 4. Metrics
acc_A_final = evaluate(model_current, test_A, DEVICE)
acc_B_final = evaluate(model_current, test_B, DEVICE)
forgetting = acc_A_initial - acc_A_final

results.append({
    "Method": "Magnitude Pruning",
    "Task A Acc (Retention)": acc_A_final,
    "Task B Acc (Plasticity)": acc_B_final,
    "Forgetting": forgetting,
    "Prep Time (s)": prep_time
})
print(f"Magnitude Pruning Result: Retention={acc_A_final:.2f}%, Forgetting={forgetting:.2f}%")

In [ ]:
# --- CELL 8: Final Analysis ---
print("\n--- Final Results ---")
df = pd.DataFrame(results)
print(df)

# Visualization [cite: 87]
if not df.empty:
    df.plot(x="Method", y=["Task A Acc (Retention)", "Forgetting"], kind="bar")
    plt.title("Catastrophic Forgetting Analysis")
    plt.ylabel("Accuracy / Drop")
    plt.tight_layout()
    plt.show()
else:
    print("No results to visualize yet. Run at least one experiment cell.")